In [3]:
import numpy as np
import pandas as pd
from os.path import join

In [4]:
DATA_DIR = ''

In [5]:
#get ccle data downloaded from https://data.broadinstitute.org/ccle/CCLE_RNAseq_rsem_genes_tpm_20180929.txt.gz
ccle_df = pd.read_csv(join(DATA_DIR, 'CCLE/CCLE_RNAseq_rsem_genes_tpm_20180929.txt'), sep='\t', index_col=0)

In [6]:

def get_ccle_data(ccle_df):
    
    
    #remove trailing dot (.)
    ccle_df.index = ccle_df.index.str.replace('\.[^.]*$','')
    #get coding genes
    hgnc_coding_genes_df = pd.read_csv(join(DATA_DIR, 'genes/HUGO_genes/gene_with_protein_product.txt'), sep='\t')
    #remove genes with Ensembl ID
    hgnc_coding_genes_df = hgnc_coding_genes_df.dropna(subset=['ensembl_gene_id'])
    
    #filter using coding genes
    ensembl_gene_id = list(hgnc_coding_genes_df.ensembl_gene_id.unique())
    ccle_filtered_df = ccle_df.reindex(index = ensembl_gene_id)
    
    #change Ensembl ID to HGNC symbol of each gene
    renaming_df = hgnc_coding_genes_df[['ensembl_gene_id', 'symbol']]
    renaming_df = renaming_df.set_index('ensembl_gene_id')
    ccle_filtered_df = renaming_df.join(ccle_filtered_df, how='inner')
    ccle_filtered_df = ccle_filtered_df.set_index('symbol')
    ccle_filtered_df.drop('transcript_ids', axis=1, inplace=True)
    return ccle_filtered_df
    
    

In [7]:
ccle_df2 = get_ccle_data(ccle_df)
ccle_df2.head()

/Users/haithamelmarakeby/anaconda2/envs/tftorch/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2818: DtypeWarning: Columns (38,45) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,22RV1_PROSTATE,2313287_STOMACH,253JBV_URINARY_TRACT,253J_URINARY_TRACT,42MGBA_CENTRAL_NERVOUS_SYSTEM,5637_URINARY_TRACT,59M_OVARY,639V_URINARY_TRACT,647V_URINARY_TRACT,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,...,WSUDLCL2_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,YAPC_PANCREAS,YD10B_UPPER_AERODIGESTIVE_TRACT,YD15_SALIVARY_GLAND,YD38_UPPER_AERODIGESTIVE_TRACT,YD8_UPPER_AERODIGESTIVE_TRACT,YH13_CENTRAL_NERVOUS_SYSTEM,YKG1_CENTRAL_NERVOUS_SYSTEM,ZR751_BREAST,ZR7530_BREAST
symbol,,,,,,,,,,,,,,,,,,,,,
A1BG,2.30,0.14,0.43,1.06,49.81,0.18,30.81,31.30,1.52,41.26,...,73.59,0.00,0.00,0.61,0.34,1.20,27.57,21.04,47.74,33.13
A1CF,8.59,3.47,0.03,0.06,0.03,0.00,0.01,0.00,0.00,0.00,...,0.00,0.02,0.26,0.00,0.00,0.01,0.01,0.00,0.06,0.00
A2M,2.99,0.06,0.18,0.04,0.12,0.10,0.28,0.01,0.06,0.17,...,0.32,0.20,0.22,0.20,0.12,2.54,5.00,0.34,0.18,0.87
A2ML1,0.11,0.08,0.05,0.01,0.00,2.61,0.05,0.01,0.01,0.01,...,0.01,0.07,0.17,0.22,0.49,0.01,0.01,0.01,0.00,0.71
A3GALT2,0.00,0.00,0.00,0.00,0.08,0.00,0.03,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.19,0.00,0.00


In [8]:
ccle_df2.dropna(how='all', inplace=True)

In [9]:
ccle_df2.to_csv('ccle_protein_coding_genes.csv')

In [10]:
def get_cancer_genes():
    cancer_genes_df = pd.read_csv(join(DATA_DIR, 'genes/Census_allWed Feb  6 15_02_31 2019.csv'), index_col =0 )
    cancer_genes = list(cancer_genes_df.index.unique())
    return cancer_genes
    

In [11]:
cancer_genes = get_cancer_genes()

In [12]:
ccle_cancer_genes_df = ccle_df2.reindex(cancer_genes)

In [13]:
ccle_cancer_genes_df.dropna(how='all', inplace=True)
ccle_cancer_genes_df.shape
ccle_cancer_genes_df.to_csv('ccle_cancer_genes.csv')